# RES Timeseries Data Extraction


This Jupyter Notebook downloads and cleans up solar and wind timeseries from the German TSOs TransnetBW and Tennet.

## Loading some python libraries needed later

In [1]:
import urllib
import os
import pandas as pd
import numpy as np
import io

## Configuring URLs

In [2]:
conf = {
    'transnetbw': {
        'pv':  "https://www.transnetbw.de/de/kennzahlen/erneuerbare-energien/fotovoltaik?app=solar&activeTab=csv&selectMonatDownload={month}&view=1&download=true",
        'wind':  'https://www.transnetbw.de/de/kennzahlen/erneuerbare-energien/windenergie?app=wind&activeTab=csv&selectMonatDownload={month}&view=1&download=true',
    }
    ,'tennet': {
        'pv':  "http://www.tennettso.de/site/de/phpbridge?commandpath=Tatsaechliche_und_prognostizierte_Solarenergieeinspeisung%2FmonthDataSheetCsv.php&sub=total&querystring=monat%3D{year}-{month:02d}&contenttype=text%2Fx-csv",
        'wind':  "http://www.tennettso.de/site/de/phpbridge?commandpath=Tatsaechliche_und_prognostizierte_Windenergieeinspeisung%2FmonthDataSheetCsv.php&querystring=monat%3D{year}-{month:02d}&contenttype=text%2Fx-csv",
    },
}

In [3]:
def urls(years, months, url_templates):
    for year in years:
        for month in months:
            for url in url_templates:
                yield url.format(**{'year': year, 'month': month})
                                        
def read_content_from_url(urls):
    for url in urls:
        yield io.BytesIO(urllib.request.urlopen(url).read())
        
def read_csvs(csvs):
    i = 0
    for csv in csvs:
        i += 1
        yield pd.read_csv(
            csv,
            sep=";",
            skiprows=4,
            names=['datum','position','forecast_' + str(i),'actual_' + str(i)],
            usecols=[0, 1, 2, 3]
        )

def fill_na(data_sets):
    for data_set in data_sets:
        data_set.fillna(method='ffill',inplace=True)
        yield data_set

def extract_time(data_sets):
    for data_set in data_sets:
        data_set['hour'] = (np.trunc((data_set['position']-1) /4)).astype(int).astype(str)
        data_set['minute'] = (((data_set['position']-1) % 4)*15).astype(int).astype(str)
        data_set['time'] = data_set['datum']+' '+data_set['hour']+':'+data_set['minute']
        data_set['Timestamp'] = pd.to_datetime(data_set['time'])
        data_set.set_index('Timestamp',inplace=True)
        del data_set['position']
        del data_set['hour'] 
        del data_set['minute'] 
        del data_set['datum'] 
        del data_set['time'] 
        yield data_set

In [4]:
tennet_urls = urls([2015], [1, 2], conf['tennet'].values())
transnetbw_urls = urls([2015], range(1, 25), conf['transnetbw'].values())

In [5]:
tennet_sets = extract_time(fill_na(read_csvs(read_content_from_url(tennet_urls))))
#transnetbw_sets = delete_columns(extract_time(read_csvs(read_content_from_url(urls([2015], range(1, 25), conf['transnetbw'].values())))))

In [6]:
all_data_sets = pd.DataFrame()
for data_set in tennet_sets:
    all_data_sets = all_data_sets.combine_first(data_set)

In [7]:
all_data_sets

,actual_1,actual_2,actual_3,actual_4,forecast_1,forecast_2,forecast_3,forecast_4
Timestamp,,,,,,,,
2015-01-01 00:00:00,4621,0,NaN,NaN,4091,0,NaN,NaN
2015-01-01 00:15:00,4549,0,NaN,NaN,4182,0,NaN,NaN
2015-01-01 00:30:00,4487,0,NaN,NaN,4273,0,NaN,NaN
2015-01-01 00:45:00,4457,0,NaN,NaN,4365,0,NaN,NaN
2015-01-01 01:00:00,4403,0,NaN,NaN,4427,0,NaN,NaN
2015-01-01 01:15:00,4357,0,NaN,NaN,4456,0,NaN,NaN
2015-01-01 01:30:00,4295,0,NaN,NaN,4484,0,NaN,NaN
2015-01-01 01:45:00,4352,0,NaN,NaN,4513,0,NaN,NaN
2015-01-01 02:00:00,4384,0,NaN,NaN,4534,0,NaN,NaN


## Downloading the data

Here we loop through the configuration defined above by TSO (transnetbw, tennet) and Technology (pv, wind).

We then do slightly different things depending on which TSO we're in, because Tennet has month and year as variables in the URL whereas TransnetBW only has the month in there.

In [ ]:


# Download data

for tsoName, tsoConf in conf.iteritems():
    for techName, url in tsoConf.iteritems():
        
        if tsoName == 'tennet':
            for year in range (2014, 2015): # This means a range from 2014 to 2014 (including)
                path = 'csv/'+tsoName+'/'+techName+'/'+str(year)+'/'
                if not os.path.exists(path): os.makedirs(path)

                for month in range(1, 13): # This means a range from 1 to 12 (including)
                    full_url = url % {'month': month, 'year': year}
                    urllib.urlretrieve(full_url, path+"%d.csv" % month)

        if tsoName == 'transnetbw':
            path = 'csv/'+tsoName+'/'+techName+'/'
            if not os.path.exists(path): os.makedirs(path)
                
            for month in range(1, 25): # This means the last 24 months
                full_url = url % month
                urllib.urlretrieve(full_url, path+"%d.csv" % month)




## Define individual read functions

The TSOs have different columns and date formats they use in their CSV files. Here we define individual functions for the two TSOs to read-in a single monthly file in their specific format.

We're going to use these functions in the loop for all months later on.

In [ ]:
# Define specific readData functions for the TSOs

def readData_tennet(filePath, tsoName, techName):
    data = pandas.read_csv(
        filePath,
        sep=";",
        skiprows=4,
        names=['datum','position','forecast_'+tsoName+'_'+techName,'actual_'+tsoName+'_'+techName,'unknown']
    )

    data.fillna(method='ffill',inplace=True)

    data['hour'] = (np.trunc((data['position']-1) /4)).astype(int).astype(str)
    data['minute'] = (((data['position']-1) % 4)*15).astype(int).astype(str)
    data['time'] = data['datum']+' '+data['hour']+':'+data['minute']

    data['Timestamp'] = pandas.to_datetime(data['time'])

    data.set_index('Timestamp',inplace=True)

    del data['position']
    del data['hour'] 
    del data['minute'] 
    del data['datum'] 
    del data['time'] 
    del data['unknown']
    return data

                
def readData_transnet(filePath, tsoName, techName):
    data = pandas.read_csv(
        filePath,
        decimal=',',
        sep=";",
        parse_dates = {'Timestamp' : ['Datum von', 'Uhrzeit von']},
        index_col = "Timestamp",
        dayfirst=True,
    )

    data = data.drop('Unnamed: 6', 1)
    data = data.drop('Datum bis', 1)
    data = data.drop('Uhrzeit bis', 1)
    data.fillna(0,inplace=True)
    
    forecastCol = 'forecast_'+tsoName+'_'+techName
    actualCol = 'actual_'+tsoName+'_'+techName

    data.rename(
        columns={'Prognose (MW)': forecastCol, 'Ist-Wert (MW)': actualCol},
        inplace=True
    )
    

    if 'Datum' in data.columns: 
        del data['Datum']
    if 'Uhrzeit' in data.columns: 
        del data['Uhrzeit']    
    
    return data




## Testing the read functions

If you want to test the individual read functions just for checking if everything works, you can uncomment the line below.

In [ ]:
# transnetTestDataWind = readData_transnet("csv/transnetbw/wind/7.csv", 'transnetbw', 'wind')

# And now output the TestData:
# transnetTestDataWind

## Loading the data into memory and cleaning it up



In [ ]:

# Read and merge data

resultDataSet = pandas.DataFrame();

for tsoName, tsoConf in conf.iteritems():
    for techName, url in tsoConf.iteritems():

        if tsoName == 'tennet':
            for year in range (2014, 2015):
                for month in range(1, 12):
                    filePath = 'csv/'+tsoName+'/'+techName+'/'+str(year)+'/'+str(month)+'.csv'
                    try:
                        df = readData_tennet(filePath, tsoName, techName)
                        try:
                            df = readData_tennet(filePath, tsoName, techName)
                            resultDataSet = resultDataSet.combine_first(df)
                        except TypeError:
                            atLeastOneError = 'yes'
                    except ValueError:
                        atLeastOneError = 'yes'

        if tsoName == 'transnetbw':
            for month in range(1, 20):
                filePath = 'csv/'+tsoName+'/'+techName+'/'+str(month)+'.csv'
                df = readData_transnet(filePath, tsoName, techName)
                try:
                    df = readData_transnet(filePath, tsoName, techName)
                    resultDataSet = resultDataSet.combine_first(df)
                except ValueError:
                    atLeastOneError = 'yes'
                    print 'error'


if 'Datum' in resultDataSet.columns: 
    del resultDataSet['Datum']
if 'Uhrzeit' in resultDataSet.columns: 
    del resultDataSet['Uhrzeit']                    


## Validation and Plausibility Checks

Ideally you would do plausibility checks on your data here. (Check for missing data, check that output is never negative, etc.)

In [ ]:
# Here you would do plausibility checks...

## Save to one big CSV file

In [ ]:
resultDataSet.to_csv("csv/allData.csv")

## Display the data

Something seems to be broken with the Tennet wind data... Other than that, the time spans of the different TSO datasets read-in above are different.

In [ ]:
resultDataSet

## Example: Resample the timeseries data to 60minute

Here we use Pandas' powerful timeseries functionality to resample the data to 60minute intervals 

In [ ]:
resultDataSet.resample('60Min',how='sum')